In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as si
import utils as us
from Cluster_Ensembles import CSPA, HGPA, MCLA
import matplotlib.pyplot as plt
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from sklearn.metrics.cluster import adjusted_rand_score as ARI
from cluster_ensembles_sample import cluster_ensembles
import joblib
import warnings
import random
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelBinarizer
from coclust.clustering.spherical_kmeans import SphericalKmeans
from coclust.coclustering import CoclustMod,CoclustSpecMod, CoclustInfo
from coclust.clustering.spherical_kmeans import SphericalKmeans
from coclust.visualization import (plot_reorganized_matrix,
                                  plot_cluster_top_terms,
                                  plot_max_modularities)

In [2]:
db_names = ['re0', 'fbis', 'wap' ]
data = us.load_data(db_names,dir_path='../cluto')

## Table des donnees

In [3]:
data_table = us.create_data_table(data)
data_table

,dataset,# document,# mots,# class,sparsité,pt class/ gr class
0,re0,1504,2886,13,0.982,0.018
1,fbis,2463,2000,17,0.920,0.075
2,wap,1560,8460,20,0.983,0.015


In [106]:
def find_asso_nmi_ari(algo, pred_lab, reel_lab,):

    association = us.total_association(pred_lab)
    algo.fit(association)
    asso_result = [NMI(reel_lab,algo.row_labels_), ARI(reel_lab, algo.row_labels_)]
    
    return asso_result

def consensus_nmi_ari(pred_labels, n,methods, reel_lab):
    ans_ = []
    scores = []
    
    for method in methods:
        print(method)
        score1,ans1 = cluster_ensembles(pred_labels,False,n,method)
        ans_.append(ans1)
        scores.append(score1)
    ind = np.array(scores).argsort()[-1:]
    ind = ind[0]
    
    consen_result = [NMI(reel_lab,ans_[ind]), ARI(reel_lab, ans_[ind])]
    return consen_result
    
def concat(lab1,lab2):
    lab1 = list(lab1)
    lab2 = list(lab2)
    for lab in lab2:
        lab1.append(lab)
    return np.array(lab1)


def sk_best_partitions(data,k,n=5,runs=15):
    
    partitions = []
    
    for i in range(runs):
        sk = SphericalKmeans(k)
        sk.fit(data)
        partitions.append(sk)
        
    best_index = [ part.criterion for part in partitions]

    ind = np.array(best_index).argsort()[-n:]
    best_part = np.take(partitions, ind)
    
    row_lab = np.array([item.labels_ for item in best_part])
    
    return row_lab

def find_partitions(data, algo,minimized,method="CI", n=5,  runs=15):

    results = []
    labs = []
    for _ in range(runs):
        copy_algo = algo
        copy_algo.fit(data)
        labs.append(np.array(copy_algo.row_labels_))
        results.append(copy_algo)

    # joblib.dump(results, f'./{name}.joblib')

    if method == "CI":
        vals = [part.criterion for part in results]
    elif method == "CM":
        vals = [part.modularity for part in results]
    else:
        vals = [random.random() for _ in results]

    if minimized:
        ind = np.array(vals).argsort()[:n]
    else:
        ind = np.array(vals).argsort()[-n:]
    
    best_part = np.take(labs, ind)
#     print([print(item.row_labels_) for item in best_part])
#     row_labels = np.array([item.row_labels_ for item in best_part])
    
    return best_part, labs


## Spherical Kmeans 

In [7]:
re0_sk_res =  sk_best_partitions(data['re0'][0],13)

fbis_sk_res =  sk_best_partitions(data['fbis'][0],17)

wap_sk_res =  sk_best_partitions(data['wap'][0],20)

## - CoclustInfo 

#### Execution

In [9]:
algo = CoclustInfo(n_row_clusters=13,n_col_clusters=13)
re0_co_inf_res = us.find_partitions(data['re0'][0],algo,True)

algo = CoclustInfo(n_row_clusters=17,n_col_clusters=17)
fbis_co_inf_res = us.find_partitions(data['fbis'][0],algo,True)

algo = CoclustInfo(n_row_clusters=20,n_col_clusters=20)
wap_co_inf_res = us.find_partitions(data['wap'][0],algo,True)

### cluster_ensemble

In [11]:
re0_co_inf_consence = consensus_nmi_ari(re0_co_inf_res,13,[CSPA, HGPA, MCLA], data['re0'][1])

fbis_co_inf_consence = consensus_nmi_ari(fbis_co_inf_res,17,[CSPA, HGPA, MCLA], data['fbis'][1])

wap_co_inf_consence = consensus_nmi_ari(wap_co_inf_res,20,[CSPA, HGPA, MCLA], data['wap'][1])

In [12]:
print(re0_co_inf_consence)
print(fbis_co_inf_consence)
print(wap_co_inf_consence)

[0.38419917666162623, 0.1585011665999189]
[0.5252365798904505, 0.2857792419724781]
[0.5058249532096252, 0.2548498492476764]


### Co-association

In [13]:
algo = CoclustInfo(n_row_clusters=13, n_col_clusters=13)
re0_co_inf_associ = find_asso_nmi_ari(algo, re0_co_inf_res, data['re0'][1])

algo = CoclustInfo(n_row_clusters=17, n_col_clusters=17)
fbis_co_inf_associ = find_asso_nmi_ari(algo, fbis_co_inf_res, data['fbis'][1])

algo = CoclustInfo(n_row_clusters=20, n_col_clusters=20)
wap_co_inf_associ = find_asso_nmi_ari(algo, wap_co_inf_res, data['wap'][1])

In [14]:
print(re0_co_inf_associ)
print(fbis_co_inf_associ)
print(wap_co_inf_associ)

[0.2825362876079906, 0.101362554821126]
[0.4464803605986396, 0.28694291307969677]
[0.438614555264813, 0.30153803160674014]


### cluster_ensemble with skmeans

In [16]:
re0_co_inf_sk_consence = consensus_nmi_ari(concat(re0_co_inf_res, re0_sk_res),13,[CSPA, HGPA, MCLA], data['re0'][1])

fbis_co_inf_sk_consence = consensus_nmi_ari(concat(fbis_co_inf_res, fbis_sk_res),17,[CSPA, HGPA, MCLA], data['fbis'][1])

wap_co_inf_sk_consence = consensus_nmi_ari(concat(wap_co_inf_res, wap_sk_res),20,[CSPA, HGPA, MCLA], data['wap'][1])

In [17]:
print(re0_co_inf_sk_consence,)
print(fbis_co_inf_sk_consence)
print(wap_co_inf_sk_consence)

[0.3905195031398707, 0.1769395556597831]
[0.5586682511049379, 0.3567314491954577]
[0.5291947031206292, 0.3075255009891753]


### Co-association with skmeans

In [18]:
algo = CoclustInfo(n_row_clusters=13, n_col_clusters=13)
re0_co_inf_sk_associ = find_asso_nmi_ari(algo, concat(re0_co_inf_res, re0_sk_res), data['re0'][1])

algo = CoclustInfo(n_row_clusters=17, n_col_clusters=17)
fbis_co_inf_sk_associ = find_asso_nmi_ari(algo, concat(fbis_co_inf_res, fbis_sk_res), data['fbis'][1])

algo = CoclustInfo(n_row_clusters=20, n_col_clusters=20)
wap_co_inf_sk_associ = find_asso_nmi_ari(algo, concat(wap_co_inf_res, wap_sk_res), data['wap'][1])

In [19]:
print(re0_co_inf_sk_associ)
print(fbis_co_inf_sk_associ)
print(fbis_co_inf_sk_associ)

[0.3514501666571611, 0.16617447586769862]
[0.5225593667153169, 0.3493741940816951]
[0.5225593667153169, 0.3493741940816951]


## - CoclustInfo X2

### Execution

In [21]:
algo = CoclustInfo(n_row_clusters=13,n_col_clusters=26)
re0_co_inf_x2_res = us.find_partitions(data['re0'][0],algo,True)

algo = CoclustInfo(n_row_clusters=17,n_col_clusters=34)
fbis_co_inf_x2_res = us.find_partitions(data['fbis'][0],algo,True)

algo = CoclustInfo(n_row_clusters=20,n_col_clusters=40)
wap_co_inf_x2_res = us.find_partitions(data['wap'][0],algo,True)

### cluster_ensembles

In [30]:
re0_co_inf_x2_consence = consensus_nmi_ari(re0_co_inf_x2_res,13,[CSPA, HGPA, MCLA], data['re0'][1])

fbis_co_inf_x2_consence = consensus_nmi_ari(fbis_co_inf_x2_res,17,[CSPA, HGPA, MCLA], data['fbis'][1])

wap_co_inf_x2_consence = consensus_nmi_ari(wap_co_inf_x2_res,20,[CSPA, HGPA, MCLA], data['wap'][1])

In [26]:
print(re0_co_inf_x2_consence)
print(fbis_co_inf_x2_consence)
print(wap_co_inf_x2_consence)

[0.3671074885737563, 0.1367605741917185]
[0.5287370388110759, 0.28363299023126504]
[0.5363067613304963, 0.3170760903383303]


### Co-association

In [27]:
algo = CoclustInfo(n_row_clusters=13, n_col_clusters=13)
re0_co_inf_x2_associ = find_asso_nmi_ari(algo, re0_co_inf_x2_res, data['re0'][1])

algo = CoclustInfo(n_row_clusters=17, n_col_clusters=17)
fbis_co_inf_x2_associ = find_asso_nmi_ari(algo, fbis_co_inf_x2_res, data['fbis'][1])

algo = CoclustInfo(n_row_clusters=20, n_col_clusters=20)
wap_co_inf_x2_associ = find_asso_nmi_ari(algo, wap_co_inf_x2_res, data['wap'][1])

In [28]:
print(re0_co_inf_x2_associ)
print(fbis_co_inf_x2_associ)
print(wap_co_inf_x2_associ)

[0.2472362755306274, 0.07158724777916535]
[0.40983879475324175, 0.22858569099713077]
[0.40701987775266146, 0.1820503816160997]


### cluster_ensemble with skmeans

In [31]:
re0_co_inf_x2_sk_consence = consensus_nmi_ari(concat(re0_co_inf_x2_res, re0_sk_res),13,[CSPA, HGPA, MCLA], data['re0'][1])

fbis_co_inf_x2_sk_consence = consensus_nmi_ari(concat(fbis_co_inf_x2_res, fbis_sk_res),17,[CSPA, HGPA, MCLA], data['fbis'][1])

wap_co_inf_x2_sk_consence = consensus_nmi_ari(concat(wap_co_inf_x2_res, wap_sk_res),20,[CSPA, HGPA, MCLA], data['wap'][1])

In [32]:
print(re0_co_inf_x2_sk_consence)
print(fbis_co_inf_x2_sk_consence)
print(wap_co_inf_x2_sk_consence)

[0.38024069758628903, 0.1635487814829469]
[0.5785767007411985, 0.3476564456797294]
[0.545351893464675, 0.3129515557460158]


### Co-association with skmeans

In [33]:
algo = CoclustInfo(n_row_clusters=13, n_col_clusters=13)
re0_co_inf_x2_sk_associ = find_asso_nmi_ari(algo, concat(re0_co_inf_x2_res, re0_sk_res), data['re0'][1])

algo = CoclustInfo(n_row_clusters=17, n_col_clusters=17)
fbis_co_inf_x2_sk_associ = find_asso_nmi_ari(algo, concat(fbis_co_inf_x2_res, fbis_sk_res), data['fbis'][1])

algo = CoclustInfo(n_row_clusters=20, n_col_clusters=20)
wap_co_inf_x2_sk_associ = find_asso_nmi_ari(algo, concat(wap_co_inf_x2_res, wap_sk_res), data['wap'][1])

In [34]:
print(re0_co_inf_x2_sk_associ)
print(fbis_co_inf_x2_sk_associ)
print(wap_co_inf_x2_sk_associ)

[0.29766762130079516, 0.10986274529370281]
[0.5160387068761558, 0.32813941959599074]
[0.564232146623185, 0.41445656384587076]


## - CoclustInfo x3

### Execution

In [23]:
algo = CoclustInfo(n_row_clusters=13,n_col_clusters=39)
re0_co_inf_x3_res = us.find_partitions(data['re0'][0],algo,True)

algo = CoclustInfo(n_row_clusters=17,n_col_clusters=51)
fbis_co_inf_x3_res = us.find_partitions(data['fbis'][0],algo,True)

algo = CoclustInfo(n_row_clusters=20,n_col_clusters=60)
wap_co_inf_x3_res = us.find_partitions(data['wap'][0],algo,True)

### cluster_ensembles

In [37]:
re0_co_inf_x3_consence = consensus_nmi_ari(re0_co_inf_x3_res,13,[CSPA, HGPA, MCLA], data['re0'][1])

fbis_co_inf_x3_consence = consensus_nmi_ari(fbis_co_inf_x3_res,17,[CSPA, HGPA, MCLA], data['fbis'][1])

wap_co_inf_x3_consence = consensus_nmi_ari(wap_co_inf_x3_res,20,[CSPA, HGPA, MCLA], data['wap'][1])

In [36]:
print(re0_co_inf_x3_consence)
print(fbis_co_inf_x3_consence)
print(wap_co_inf_x3_consence)

[0.3673346380882476, 0.1421527166091746]
[0.5566156882633964, 0.30391646791044374]
[0.5267553910273326, 0.3106584220484953]


### Co-association

In [38]:
algo = CoclustInfo(n_row_clusters=13, n_col_clusters=13)
re0_co_inf_x3_associ = find_asso_nmi_ari(algo, re0_co_inf_x3_res, data['re0'][1])

algo = CoclustInfo(n_row_clusters=17, n_col_clusters=17)
fbis_co_inf_x3_associ = find_asso_nmi_ari(algo, fbis_co_inf_x3_res, data['fbis'][1])

algo = CoclustInfo(n_row_clusters=20, n_col_clusters=20)
wap_co_inf_x3_associ = find_asso_nmi_ari(algo, wap_co_inf_x3_res, data['wap'][1])

In [39]:
print(re0_co_inf_x3_associ)
print(fbis_co_inf_x3_associ)
print(wap_co_inf_x3_associ)

[0.2627135513574062, 0.12572374728014477]
[0.4480313791166249, 0.2608175411082891]
[0.3974522309217705, 0.28654955834084866]


### cluster_ensembles with skmeans

In [41]:
re0_co_inf_x3_sk_consence = consensus_nmi_ari(concat(re0_co_inf_x3_res, re0_sk_res),13,[CSPA, HGPA, MCLA], data['re0'][1])

fbis_co_inf_x3_sk_consence = consensus_nmi_ari(concat(fbis_co_inf_x3_res, fbis_sk_res),17,[CSPA, HGPA, MCLA], data['fbis'][1])

wap_co_inf_x3_sk_consence = consensus_nmi_ari(concat(wap_co_inf_x3_res, wap_sk_res),20,[CSPA, HGPA, MCLA], data['wap'][1])

In [42]:
print(re0_co_inf_x3_sk_consence)
print(fbis_co_inf_x3_sk_consence)
print(wap_co_inf_x3_sk_consence)

[0.39423485450890816, 0.1849156745069482]
[0.5892672939254497, 0.40915543054519043]
[0.5422570723749323, 0.3351236612717501]


### Co-association with skmeans

In [43]:
algo = CoclustInfo(n_row_clusters=13, n_col_clusters=13)
re0_co_inf_x3_sk_associ = find_asso_nmi_ari(algo, concat(re0_co_inf_x3_res, re0_sk_res), data['re0'][1])

algo = CoclustInfo(n_row_clusters=17, n_col_clusters=17)
fbis_co_inf_x3_sk_associ = find_asso_nmi_ari(algo, concat(fbis_co_inf_x3_res, fbis_sk_res), data['fbis'][1])

algo = CoclustInfo(n_row_clusters=20, n_col_clusters=20)
wap_co_inf_x3_sk_associ = find_asso_nmi_ari(algo, concat(wap_co_inf_x3_res, wap_sk_res), data['wap'][1])

In [44]:
print(re0_co_inf_x3_sk_associ)
print(fbis_co_inf_x3_sk_associ)
print(wap_co_inf_x3_sk_associ)

[0.38247278243742877, 0.19659917015390846]
[0.5554193440762294, 0.39821971500959275]
[0.5571967659546535, 0.4132695701912138]


## CoclustMod

### Execution

In [114]:
algo = CoclustMod(n_clusters=13)
re0_co_mod_res, temp1 = find_partitions(data['re0'][0],algo,False, method="CM")

algo = CoclustMod(n_clusters=17)
fbis_co_mod_res, temp2 = find_partitions(data['fbis'][0],algo,False, method="CM")

algo = CoclustMod(n_clusters=20)
wap_co_mod_res, temp3 = find_partitions(data['wap'][0],algo,False, method="CM")

In [149]:
re0_co_mod_res = np.array([temp1[0], temp1[0]])

In [150]:
fbis_co_mod_res = np.array([temp2[0], temp2[1],temp2[5]])

In [151]:
wap_co_mod_res = np.array([temp3[0], temp3[9]])

### cluster_ensembles

In [153]:
re0_co_mod_consence = consensus_nmi_ari(re0_co_mod_res,13,[CSPA, HGPA, MCLA], data['re0'][1])

fbis_co_mod_consence = consensus_nmi_ari(fbis_co_mod_res,17,[CSPA, HGPA, MCLA], data['fbis'][1])

wap_co_mod_consence = consensus_nmi_ari(wap_co_mod_res,20,[CSPA, HGPA, MCLA], data['wap'][1])

In [154]:
print(re0_co_mod_consence)
print(fbis_co_mod_consence)
print(wap_co_mod_consence)

[0.3453663681631249, 0.18219986375764172]
[0.40390568444414676, 0.23961386151011263]
[0.4625641717271864, 0.33606224676979757]


### Co-association

In [155]:
algo = CoclustMod(n_clusters=13)
re0_co_mod_associ = find_asso_nmi_ari(algo, re0_co_mod_res, data['re0'][1])

algo = CoclustMod(n_clusters=17)
fbis_co_mod_associ = find_asso_nmi_ari(algo, fbis_co_mod_res, data['fbis'][1])

algo = CoclustMod(n_clusters=20)
wap_co_mod_associ = find_asso_nmi_ari(algo, wap_co_mod_res, data['wap'][1])

In [156]:
print(re0_co_mod_associ)
print(fbis_co_mod_associ)
print(wap_co_mod_associ)

[0.29846765955816285, 0.12006697686844824]
[0.46462479228766673, 0.3160282343495924]
[0.46464365121575346, 0.396146187764163]


### cluster_ensembles with skmeans

In [160]:
re0_co_mod_sk_consence = consensus_nmi_ari(concat(re0_co_mod_res, re0_sk_res),13,[CSPA, HGPA, MCLA], data['re0'][1])

fbis_co_mod_sk_consence = consensus_nmi_ari(concat(fbis_co_mod_res, fbis_sk_res),17,[CSPA, HGPA, MCLA], data['fbis'][1])

wap_co_mod_sk_consence = consensus_nmi_ari(concat(wap_co_mod_res, wap_sk_res),20,[CSPA, HGPA, MCLA], data['wap'][1])

In [161]:
print(re0_co_mod_sk_consence)
print(fbis_co_mod_sk_consence)
print(wap_co_mod_sk_consence)

[0.41088261299344186, 0.2155053878763645]
[0.5766892506980553, 0.3961320776540335]
[0.5622439196501479, 0.3399706824186719]


### Co-association with skmeans

In [162]:
algo = CoclustMod(n_clusters=13)
re0_co_mod_sk_associ = find_asso_nmi_ari(algo, concat(re0_co_mod_res, re0_sk_res), data['re0'][1])

algo = CoclustMod(n_clusters=17)
fbis_co_mod_sk_associ = find_asso_nmi_ari(algo, concat(fbis_co_mod_res, fbis_sk_res), data['fbis'][1])

algo = CoclustMod(n_clusters=20)
wap_co_mod_sk_associ = find_asso_nmi_ari(algo, concat(wap_co_mod_res, wap_sk_res), data['wap'][1])

In [163]:
print(re0_co_mod_sk_associ)
print(fbis_co_mod_sk_associ)
print(wap_co_mod_sk_associ)

[0.3599625696423616, 0.18042618260308751]
[0.5595482728761252, 0.3635455975759135]
[0.580169694302202, 0.469368419005595]


## ColcustSpecMod

### Execution

In [59]:
algo = CoclustSpecMod(n_clusters=13)
re0_sp_mod_res = find_partitions(data['re0'][0],algo,False, method="CSM")

algo = CoclustSpecMod(n_clusters=17)
fbis_sp_mod_res = us.find_partitions(data['fbis'][0],algo,False, method="CSM")

algo = CoclustSpecMod(n_clusters=20)
wap_sp_mod_res = us.find_partitions(data['wap'][0],algo,False, method="CSM")

### cluster_ensembles

In [83]:
re0_sp_mod_consence = consensus_nmi_ari(re0_sp_mod_res,13,[CSPA, HGPA, MCLA], data['re0'][1])

fbis_sp_mod_consence = consensus_nmi_ari(fbis_sp_mod_res,17,[CSPA, HGPA, MCLA], data['fbis'][1])

wap_sp_mod_consence = consensus_nmi_ari(wap_sp_mod_res,20,[CSPA, HGPA, MCLA], data['wap'][1])

In [82]:
print(re0_sp_mod_consence)
print(fbis_sp_mod_consence)
print(wap_sp_mod_consence)

[0.29382874196187836, 0.07792914485718451]
[0.4598144077140665, 0.30076436995212374]
[0.443629773046578, 0.22246731883287224]


### Co-association

In [84]:
algo = CoclustSpecMod(n_clusters=13)
re0_sp_mod_associ = find_asso_nmi_ari(algo, re0_sp_mod_res, data['re0'][1])

algo = CoclustSpecMod(n_clusters=17)
fbis_sp_mod_associ = find_asso_nmi_ari(algo, fbis_sp_mod_res, data['fbis'][1])

algo = CoclustSpecMod(n_clusters=20)
wap_sp_mod_associ = find_asso_nmi_ari(algo, wap_sp_mod_res, data['wap'][1])

In [85]:
print(re0_sp_mod_associ)
print(fbis_sp_mod_associ)
print(wap_sp_mod_associ)

[0.36289981257565995, 0.1491835890495418]
[0.4914220305070455, 0.30026164019673707]
[0.538163865163268, 0.3511565648120293]


### cluster_ensembles with skmeans

In [88]:
re0_sp_mod_sk_consence = consensus_nmi_ari(concat(re0_sp_mod_res, re0_sk_res),13,[CSPA, HGPA, MCLA], data['re0'][1])

fbis_sp_mod_sk_consence = consensus_nmi_ari(concat(fbis_sp_mod_res, fbis_sk_res),17,[CSPA, HGPA, MCLA], data['fbis'][1])

wap_sp_mod_sk_consence = consensus_nmi_ari(concat(wap_sp_mod_res, wap_sk_res),20,[CSPA, HGPA, MCLA], data['wap'][1])

In [87]:
print(re0_sp_mod_sk_consence)
print(fbis_sp_mod_sk_consence)
print(wap_sp_mod_sk_consence)

[0.3896234857062725, 0.13404689886869184]
[0.5208266681802665, 0.37187631215588385]
[0.5170090866201392, 0.3076978930202227]


### Co-association with skmeans

In [89]:
algo = CoclustMod(n_clusters=13)
re0_sp_mod_sk_associ = find_asso_nmi_ari(algo, concat(re0_sp_mod_res, re0_sk_res), data['re0'][1])

algo = CoclustMod(n_clusters=17)
fbis_sp_mod_sk_associ = find_asso_nmi_ari(algo, concat(fbis_sp_mod_res, fbis_sk_res), data['fbis'][1])

algo = CoclustMod(n_clusters=20)
wap_sp_mod_sk_associ = find_asso_nmi_ari(algo, concat(wap_sp_mod_res, wap_sk_res), data['wap'][1])

In [90]:
print(re0_sp_mod_sk_associ)
print(fbis_sp_mod_sk_associ)
print(wap_sp_mod_sk_associ)

[0.37621839113860084, 0.19257636878177808]
[0.49861610949602864, 0.356261894539492]
[0.5011010664685605, 0.3930926777459356]


## Cluster_ensembles Of All

In [164]:
re0_conbine = concat( concat(re0_sp_mod_res, re0_sk_res), concat(re0_co_inf_res, re0_co_mod_res) )
fbis_conbine = concat( concat(fbis_sp_mod_res, fbis_sk_res), concat(fbis_co_inf_res, fbis_co_mod_res) )
wap_conbine = concat( concat(wap_sp_mod_res, wap_sk_res), concat(wap_co_inf_res, wap_co_mod_res) )

In [166]:

re0_all_consence = consensus_nmi_ari(re0_conbine,13,[CSPA, HGPA, MCLA], data['re0'][1])

fbis_all_consence = consensus_nmi_ari(fbis_conbine,17,[CSPA, HGPA, MCLA], data['fbis'][1])

wap_all_consence = consensus_nmi_ari(wap_conbine,20,[CSPA, HGPA, MCLA], data['wap'][1])

In [167]:
print(re0_all_consence)
print(fbis_all_consence)
print(wap_all_consence)

[0.3839216617807247, 0.17274257248885375]
[0.5005182484133436, 0.31078737336343776]
[0.5452026181225641, 0.3573431674719953]
